DATA CLEANER

In [ ]:
import pandas as pd

# 1. Load dataset
df = pd.read_csv("nyc_traffic_raw.csv")   # change filename to yours

# 2. Keep only useful columns
df = df[["LINK_ID", "BOROUGH", "DATA_AS_OF", "SPEED", "TRAVEL_TIME", "LINK_NAME"]]

# 3. Clean timestamp
df["DATA_AS_OF"] = pd.to_datetime(df["DATA_AS_OF"])

# 4. Sort by road & time
df = df.sort_values(by=["LINK_ID", "DATA_AS_OF"])

# 5. Create time features
df["hour"] = df["DATA_AS_OF"].dt.hour
df["day_of_week"] = df["DATA_AS_OF"].dt.dayofweek   # 0=Mon, 6=Sun
df["date"] = df["DATA_AS_OF"].dt.date

# 6. Generate lag features (last 12h speeds per road)
for lag in range(1, 13):
    df[f"lag_{lag}h"] = df.groupby("LINK_ID")["SPEED"].shift(lag)

# 7. Create congestion ratio
# (using max observed speed per road as "free flow" approximation)
df["free_flow_speed"] = df.groupby("LINK_ID")["SPEED"].transform("max")
df["congestion_ratio"] = df["SPEED"] / df["free_flow_speed"]

# 8. Create classification labels
def label_traffic(ratio):
    if ratio > 0.8:
        return "Light"
    elif ratio > 0.5:
        return "Moderate"
    else:
        return "Heavy"

df["traffic_label"] = df["congestion_ratio"].apply(label_traffic)

# 9. Drop rows with missing values (from lags)
df = df.dropna()

# 10. Save cleaned dataset
df.to_csv("nyc_traffic_cleaned.csv", index=False)

print("✅ Preprocessing complete. Clean dataset saved as nyc_traffic_cleaned.csv")


VALIDATOR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your cleaned dataset

df = pd.read_csv("nyc_traffic_fully_cleaned.csv")
print("🔍 Starting dataset validation...")
print("="*50)

# 1. Time continuity check
print("\n⏰ Checking timestamp continuity...")
df["DATA_AS_OF"] = pd.to_datetime(df["DATA_AS_OF"])
time_gaps = df.groupby("LINK_ID")["DATA_AS_OF"].diff().value_counts().head(10)
print("Most common time gaps:\n", time_gaps)

# 2. Speed sanity check
print("\n🚦 Checking SPEED values...")
print(df["SPEED"].describe())
if (df["SPEED"] < 0).any():
    print("⚠️ Warning: Negative speeds found!")
if (df["SPEED"] > 200).any():
    print("⚠️ Warning: Extremely high speeds found (>200 km/h).")

# 3. Travel time consistency
print("\n⏳ Checking TRAVEL_TIME values...")
print(df["TRAVEL_TIME"].describe())
if (df["TRAVEL_TIME"] <= 0).any():
    print("⚠️ Warning: Non-positive travel times found!")

# 4. Congestion ratio range
print("\n📉 Checking congestion ratio...")
if "congestion_ratio" in df.columns:
    print(df["congestion_ratio"].describe())
    if (df["congestion_ratio"] < 0).any() or (df["congestion_ratio"] > 1).any():
        print("⚠️ Warning: Congestion ratio out of [0,1] range!")
else:
    print("⚠️ congestion_ratio column not found. Did you run preprocessing?")

# 5. Label balance check
print("\n📊 Checking traffic label balance...")
if "traffic_label" in df.columns:
    print(df["traffic_label"].value_counts(normalize=True))
else:
    print("⚠️ traffic_label column not found. Did you run preprocessing?")

# 6. Duplicate row check
print("\n📑 Checking duplicate rows...")
duplicates = df.duplicated(subset=["LINK_ID", "DATA_AS_OF"]).sum()
print(f"Found {duplicates} duplicate rows.")

# 7. Borough consistency check
print("\n🏙️ Checking borough consistency...")
borough_consistency = df.groupby("LINK_ID")["BOROUGH"].nunique().value_counts()
print("Boroughs per road segment:\n", borough_consistency)

print("\n✅ Validation complete.")


VALIDATION INCONSISTENCY CLEANER

In [ ]:
import pandas as pd

# -----------------------------
# 1. Load dataset
# -----------------------------
df = pd.read_csv("nyc_traffic_cleaned.csv")  # replace with your file

# -----------------------------
# 2. Remove invalid travel times
# -----------------------------
df = df[df["TRAVEL_TIME"] > 0]

# Optional: remove extreme travel times (> mean + 3*std)
upper_limit = df["TRAVEL_TIME"].mean() + 3 * df["TRAVEL_TIME"].std()
df = df[df["TRAVEL_TIME"] <= upper_limit]

# -----------------------------
# 3. Round timestamps (optional)
# -----------------------------
df["DATA_AS_OF"] = pd.to_datetime(df["DATA_AS_OF"]).dt.round("5min")

# -----------------------------
# 4. Cap congestion ratio at 1 (already mostly clean, just in case)
# -----------------------------
if "congestion_ratio" in df.columns:
    df["congestion_ratio"] = df["congestion_ratio"].clip(upper=1)

# -----------------------------
# 5. Optional: check SPEED sanity (no changes needed based on validation)
# -----------------------------
df = df[df["SPEED"] >= 0]  # remove negative speeds if any

# -----------------------------
# 6. Save cleaned dataset
# -----------------------------
df.to_csv("nyc_traffic_fully_cleaned.csv", index=False)

print("✅ Dataset fully cleaned and saved (CSV + Parquet). Ready for ML training.")


HOT ENCODING OF PLACE NAMES


In [2]:
import pandas as pd

# -----------------------------
# 1. Load cleaned dataset
# -----------------------------
df = pd.read_csv("nyc_traffic_fully_cleaned.csv")

# -----------------------------
# 2. Only one-hot encode BOROUGH
# -----------------------------
if "BOROUGH" in df.columns:
    df = pd.get_dummies(df, columns=["BOROUGH"], drop_first=True)

# -----------------------------
# 3. Save new dataset
# -----------------------------
df.to_csv("nyc_traffic_encoded.csv", index=False)

print("✅ One-hot encoding done for BOROUGH only.")
print("➡️ Saved as nyc_traffic_encoded.csv")
print("Shape after:", df.shape)



✅ One-hot encoding done for BOROUGH only.
➡️ Saved as nyc_traffic_encoded.csv
Shape after: (9614098, 27)


boolean to integer converter

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("nyc_traffic_encoded.csv")

# List of borough boolean columns
borough_cols = ['BOROUGH_Brooklyn', 'BOROUGH_Manhattan', 'BOROUGH_Queens', 'BOROUGH_Staten Island']

# Convert these boolean columns to integers
df[borough_cols] = df[borough_cols].astype(int)

# Optional: save the updated dataset
df.to_csv("your_dataset_int.csv", index=False)

print("Borough columns converted to 0/1 successfully!")


Borough columns converted to 0/1 successfully!


: 

ADDING FUTURE LABELS

In [1]:
import pandas as pd

# -----------------------------
# 1. Load dataset
# -----------------------------
df = pd.read_csv("your_dataset_int.csv")

# -----------------------------
# 2. Sort by time
# -----------------------------
df["DATA_AS_OF"] = pd.to_datetime(df["DATA_AS_OF"])
df = df.sort_values(["LINK_ID", "DATA_AS_OF"]).reset_index(drop=True)

# -----------------------------
# 3. Create FUTURE label
# -----------------------------
# Assuming your data is 5-min interval → 12 steps = 1 hour
df["future_label"] = df.groupby("LINK_ID")["traffic_label"].shift(-12)

# Drop rows where future_label is NaN (at the tail of each group)
df = df.dropna(subset=["future_label"])

# -----------------------------
# 4. Drop unnecessary columns
# -----------------------------
drop_cols = ["LINK_NAME", "date", "DATA_AS_OF"]  # not useful for model directly
df = df.drop(columns=drop_cols)

# -----------------------------
# 5. Save processed dataset
# -----------------------------
df.to_csv("nyc_traffic_processed.csv", index=False)

print("✅ Dataset transformed successfully! Saved to nyc_traffic_processed.csv")
import pandas as pd

# -----------------------------
# 1. Load dataset
# -----------------------------
df = pd.read_csv("nyc_traffic_encoded.csv")

# -----------------------------
# 2. Sort by time
# -----------------------------
df["DATA_AS_OF"] = pd.to_datetime(df["DATA_AS_OF"])
df = df.sort_values(["LINK_ID", "DATA_AS_OF"]).reset_index(drop=True)

# -----------------------------
# 3. Create FUTURE label
# -----------------------------
# Assuming your data is 5-min interval → 12 steps = 1 hour
df["future_label"] = df.groupby("LINK_ID")["traffic_label"].shift(-12)

# Drop rows where future_label is NaN (at the tail of each group)
df = df.dropna(subset=["future_label"])

# -----------------------------
# 4. Drop unnecessary columns
# -----------------------------
drop_cols = ["LINK_NAME", "date", "DATA_AS_OF"]  # not useful for model directly
df = df.drop(columns=drop_cols)

# -----------------------------
# 5. Save processed dataset
# -----------------------------
df.to_csv("final.csv", index=False)

print("✅ Dataset transformed successfully! Saved to nyc_traffic_processed.csv")


✅ Dataset transformed successfully! Saved to nyc_traffic_processed.csv
✅ Dataset transformed successfully! Saved to nyc_traffic_processed.csv


In [2]:
print("Current labels:")
print(df["traffic_label"].value_counts())

print("\nFuture labels (1h later):")
print(df["future_label"].value_counts())


Current labels:
traffic_label
Heavy       4830646
Moderate    4312592
Light        469516
Name: count, dtype: int64

Future labels (1h later):
future_label
Heavy       4830912
Moderate    4312401
Light        469441
Name: count, dtype: int64
